### 14/07/21

* Run blast on Carmen's/Ross's second round of PCR sequences

In [3]:
import os
import subprocess
os.chdir("/cluster/db/mecoulter/Ross/")

In [2]:
!ls ./results_13-7-21_run45/*.seq

./results_13-7-21_run45/SQ21-182_Ank1_19_F_A01_008.seq
./results_13-7-21_run45/SQ21-182_Ank1_19_R_B01_007.seq
./results_13-7-21_run45/SQ21-182_Ank1_52_F_E01_004.seq
./results_13-7-21_run45/SQ21-182_Ank1_52_R_F01_003.seq
./results_13-7-21_run45/SQ21-182_Ank1_56_F_C01_006.seq
./results_13-7-21_run45/SQ21-182_Ank1_56_R_D01_005.seq
./results_13-7-21_run45/SQ21-182_LRR1_52_F_G01_002.seq
./results_13-7-21_run45/SQ21-182_LRR1_52_R_H01_001.seq


In [10]:
seq_files = []
for file in os.listdir("./results_13-7-21_run45/"):
    if file.startswith("._") or file.endswith(".ab1"):
        continue
    print(file)
    seq_files.append(file)
    
    

SQ21-182_Ank1_52_R_F01_003.seq
SQ21-182_LRR1_52_R_H01_001.seq
SQ21-182_Ank1_56_F_C01_006.seq
SQ21-182_Ank1_19_F_A01_008.seq
SQ21-182_Ank1_56_R_D01_005.seq
SQ21-182_Ank1_19_R_B01_007.seq
SQ21-182_Ank1_52_F_E01_004.seq
SQ21-182_LRR1_52_F_G01_002.seq


* Run blast using the barke genome

In [12]:
genome = "/cluster/db/mecoulter/genomes/genomes/180903_Barke_Unfiltered_chloro_clean_pseudomolecules_v1.fasta"
genome_name = genome.split("/")[-1].replace(".fasta", "")


for file in seq_files:
    print(f"Running BLAST on sequence from {file}")
    file_name = "_".join(file.split("_")[1:4]) #e.g Ank1_52_R
    subprocess.run(f"blastn -task blastn-short -query ./results_13-7-21_run45/{file}  -db {genome} -outfmt 10 -out {file_name}_blast.csv", shell=True, check=True)

Running BLAST on sequence from SQ21-182_Ank1_52_R_F01_003.seq
Running BLAST on sequence from SQ21-182_LRR1_52_R_H01_001.seq
Running BLAST on sequence from SQ21-182_Ank1_56_F_C01_006.seq
Running BLAST on sequence from SQ21-182_Ank1_19_F_A01_008.seq
Running BLAST on sequence from SQ21-182_Ank1_56_R_D01_005.seq
Running BLAST on sequence from SQ21-182_Ank1_19_R_B01_007.seq
Running BLAST on sequence from SQ21-182_Ank1_52_F_E01_004.seq
Running BLAST on sequence from SQ21-182_LRR1_52_F_G01_002.seq


* We need to check - is there evidence of SNPs in the sequence that are not just sequence errors? Ie is this some Barke contamination or is this an actual gene present in 19/56 and 52 (for 52 from PCR looks real)

* To do this, we will run blast again, but produce normal blast output showing alignments (not outfmt 10). This should give an indication how many SNPs there are in the alignment

In [22]:
for file in seq_files:
    print(f"Running BLAST on sequence from {file}")
    file_name = "_".join(file.split("_")[1:4]) #e.g Ank1_52_R
    subprocess.run(f"blastn -task blastn-short -query ./results_13-7-21_run45/{file}  -db {genome} -out {file_name}_blast.txt", shell=True, check=True)

Running BLAST on sequence from SQ21-182_Ank1_52_R_F01_003.seq
Running BLAST on sequence from SQ21-182_LRR1_52_R_H01_001.seq
Running BLAST on sequence from SQ21-182_Ank1_56_F_C01_006.seq
Running BLAST on sequence from SQ21-182_Ank1_19_F_A01_008.seq
Running BLAST on sequence from SQ21-182_Ank1_56_R_D01_005.seq
Running BLAST on sequence from SQ21-182_Ank1_19_R_B01_007.seq
Running BLAST on sequence from SQ21-182_Ank1_52_F_E01_004.seq
Running BLAST on sequence from SQ21-182_LRR1_52_F_G01_002.seq


* Now go through each blast result, to make sure NLR is the top hit.

In [14]:
import pandas as pd

header_outfmt10 = ["name", "chromosome", "percent id", "match_length", "mismatches", "gap_openings", "Query start","Query end", "subject_start", "subject_end", "evalue","bitscore"]
blast = pd.read_csv("Ank1_52_R_blast.csv", header=None)
blast.columns = header_outfmt10
blast.head()


,name,chromosome,percent id,match_length,mismatches,gap_openings,Query start,Query end,subject_start,subject_end,evalue,bitscore
0,Query_1,chr3H,96.414,753,5,5,10,740,36890109,36889357,0.000000e+00,1275.0
1,Query_1,chr3H,83.988,331,50,3,401,729,44137909,44137580,9.460000e-53,214.0
2,Query_1,chr3H,91.156,147,6,1,216,355,5172331,5172185,3.470000e-46,192.0
3,Query_1,chr3H,84.706,255,38,1,475,728,44068396,44068142,1.370000e-45,190.0
4,Query_1,chr3H,91.096,146,6,1,217,355,531071438,531071293,1.370000e-45,190.0


In [15]:
blast = pd.read_csv("Ank1_52_F_blast.csv", header=None)
blast.columns = header_outfmt10
blast.head()

,name,chromosome,percent id,match_length,mismatches,gap_openings,Query start,Query end,subject_start,subject_end,evalue,bitscore
0,Query_1,chr3H,96.809,752,4,3,8,739,36889397,36890148,0.000000e+00,1310.0
1,Query_1,chr3H,84.158,303,46,2,8,309,44137608,44137909,9.040000e-50,204.0
2,Query_1,chr3H,91.156,147,6,1,355,494,5172185,5172331,3.440000e-46,192.0
3,Query_1,chr3H,91.096,146,6,1,355,493,531071293,531071438,1.360000e-45,190.0
4,Query_1,chr3H,85.088,228,34,0,8,235,44068169,44068396,3.310000e-43,182.0


* Checking alignments in Ank1_52_F_blast.txt and Ank1_52_R_blast.txt reveals large 18bp deletion compared to Barke in both F and R sequences. Other SNPs as well. Suggests sequence is indeed from line 52 and is not contamination (but would have predicted this from PCR anyway

In [16]:
blast = pd.read_csv("LRR1_52_F_blast.csv", header=None)
blast.columns = header_outfmt10
blast.head()

,name,chromosome,percent id,match_length,mismatches,gap_openings,Query start,Query end,subject_start,subject_end,evalue,bitscore
0,Query_1,chr3H,98.995,796,6,2,14,807,36885547,36886342,0.000000e+00,1505.0
1,Query_1,chr3H,90.566,53,5,0,646,698,192434322,192434270,5.420000e-08,65.9
2,Query_1,chr3H,79.878,164,33,0,99,262,192434832,192434669,2.140000e-07,63.9
3,Query_1,chr3H,91.667,36,3,0,591,626,192434370,192434335,1.300000e-02,48.1
4,Query_1,chr5H,83.792,654,103,1,44,697,584203814,584204464,1.490000e-125,456.0


In [17]:
blast = pd.read_csv("LRR1_52_R_blast.csv", header=None)
blast.columns = header_outfmt10
blast.head()

,name,chromosome,percent id,match_length,mismatches,gap_openings,Query start,Query end,subject_start,subject_end,evalue,bitscore
0,Query_1,chr3H,96.940,719,22,0,16,734,36886379,36885661,0.000000e+00,1271.0
1,Query_1,chr3H,90.566,53,5,0,163,215,192434270,192434322,5.310000e-08,65.9
2,Query_1,chr3H,91.667,36,3,0,235,270,192434335,192434370,1.200000e-02,48.1
3,Query_1,chr3H,83.333,72,12,0,599,670,192434669,192434740,1.200000e-02,48.1
4,Query_1,chr3H,100.000,21,0,0,13,33,556515769,556515789,7.700000e-01,42.1


* Alignments (LRR1_52_R_blast.txt and LRR1_52_F_blast.txt) show sequence qute similar to Barke LRR in the NLR

In [18]:


blast = pd.read_csv("Ank1_56_F_blast.csv", header=None)
blast.columns = header_outfmt10
blast.head()

,name,chromosome,percent id,match_length,mismatches,gap_openings,Query start,Query end,subject_start,subject_end,evalue,bitscore
0,Query_1,chr3H,94.595,222,9,3,12,232,36889399,36889618,5.450000e-88,331.0
1,Query_1,chr3H,84.768,151,22,1,12,161,44137610,44137760,1.530000e-23,117.0
2,Query_1,chr3H,82.955,176,29,1,12,186,44068171,44068346,9.440000e-22,111.0
3,Query_1,chr3H,86.364,44,6,0,135,178,44134975,44135018,7.300000e-01,42.1


* Very short, not sure what is wrong here:

In [19]:
!cat ./results_13-7-21_run45/SQ21-182_Ank1_56_F_C01_006.seq

ACAATANAAAGGAGACTCACAAAACATGGGGATTACTTTGGGATTACTGAAGAAGGACANCCCTCTTCCCAACATAGTTT
ATGCATTGAGATACTGGAGCAAGCTGAAAGAAGANNCCACAACTCAGAGAAGCCTCGGTTTCTCTTGCANTAGCTGGTGA
CCATCTATGTGGAAAGTTGTTGTACCCTCGAAAATCGAAGTAATGTTTACCTACCNAAGTGTCGAGTGTTCANANTAGTA
GCATGGTATGGATTTGCACATNCATNGTGTATTGCACGGNGAAGGNAGNTCTCCANGTNCCANTTCCCANNGNGCTACTN
CCCTCCTTTNCGGGGANANNGNNCGCNTTGNACTNNGANCATNGNTTTNAAAANCCCGNNATAGCCCCNGTAAGCCTAAA
AGCCANGNANGNGNNNANGGGNGTGNGANCGAAAAGCCGCTGTAACCGGNATAGCTCNGANATNANAGAAGTTNAAGNAN
GGGGNAANTTTCCGNGGNGNGNGATTAAAANNTCCCAGAGNGGAACTNATTNTGGNNAGAAGGGGNCCNNATTCNCAAAA
GAAANAGGGNNTCACAAGGNNNNANCATGCGNAAAGNGGCGNAGGANNGGACNGAANNGANGGGAAGGANNNNAGGGNAN
GGGGNANGAGGAGAAAGGCGAGGNAAAGAGNGACNNNGGAGAGCNANNNANAAANAGCANNCNAGAGGTGCCACNCCNNA
NNNGGAANGGCGNACNAAAAAAAAAGAAAAGCNNGCNGNGGGCGNGGNNGAGGGNANAGAGNGANAGNGAGNCAANNGGN
GNGGGAANCGGACCAGCNGAGNGNANACAANAAAAANANAAAAGGACGA


In [23]:

blast = pd.read_csv("Ank1_56_R_blast.csv", header=None)
blast.columns = header_outfmt10
blast.head()

,name,chromosome,percent id,match_length,mismatches,gap_openings,Query start,Query end,subject_start,subject_end,evalue,bitscore
0,Query_1,chr3H,93.478,46,2,1,63,108,36890049,36890005,8.210000e-07,61.9
1,Query_1,chr3H,90.698,43,3,1,63,105,44068584,44068543,1.200000e-02,48.1
2,Query_1,chr3H,90.698,43,3,1,63,105,44138020,44137979,1.200000e-02,48.1
3,Query_1,chr4H,100.000,20,0,0,190,209,315526524,315526543,3.000000e+00,40.1


* Sequence weak with lots of Ns

In [20]:
blast = pd.read_csv("Ank1_19_F_blast.csv", header=None)
blast.columns = header_outfmt10
blast.head()

,name,chromosome,percent id,match_length,mismatches,gap_openings,Query start,Query end,subject_start,subject_end,evalue,bitscore
0,Query_1,chr3H,85.841,565,65,15,21,583,36889414,36889965,1.460000e-125,456.0
1,Query_1,chr3H,82.667,150,24,2,358,507,608190250,608190103,9.980000e-22,111.0
2,Query_1,chr3H,83.333,114,18,1,358,471,225505988,225505876,9.260000e-16,91.7
3,Query_1,chr3H,84.375,96,14,1,376,471,586887600,586887694,8.920000e-13,81.8
4,Query_1,chr3H,82.979,94,15,1,359,452,5172185,5172277,8.590000e-10,71.9


In [21]:
blast = pd.read_csv("Ank1_19_R_blast.csv", header=None)
blast.columns = header_outfmt10
blast.head()

,name,chromosome,percent id,match_length,mismatches,gap_openings,Query start,Query end,subject_start,subject_end,evalue,bitscore
0,Query_1,chr3H,86.395,147,15,5,53,195,36890049,36889904,9.140000e-19,101.0
1,Query_1,chr3H,95.238,42,2,0,53,94,44068584,44068543,3.230000e-09,69.9
2,Query_1,chr3H,95.238,42,2,0,53,94,44138020,44137979,3.230000e-09,69.9
3,Query_1,chr3H,100.000,22,0,0,237,258,26517470,26517491,1.800000e-01,44.1
4,Query_1,chr3H,100.000,22,0,0,237,258,519616398,519616419,1.800000e-01,44.1


* Checked Ank1_19_R_blast.txt and Ank1_19_F_blast.txt, lots of mismatches and shortened sequence matches

In [24]:
!cat ./results_13-7-21_run45/SQ21-182_Ank1_19_R_B01_007.seq

NTTTTGTCATCTTTGGAAGAATCAGCGTAACTCCGAGTATGATCNCTTGAGCATGTCCATTGCTACTCTTGCCTCNGTTG
GAAACATTATCCTCATCACCCCACTCATNGCAAGGAAAATATTGAGTGAAAAACATCCTTTCCAAGGAGAGAAAAGGTAC
TCCCTTCGCNTCAAAATNAGTGTCTGGTTCTAGTCCNATGGTTTNAAATNGNNGGCTNCGGGAAAATGNNGCGGANCCTC
TAAATCAGCTATAGCGGAAGCTANACGCGGCTANAGCGGCNAAATTGTNCATNANCCNNTNAACGGCANNTTGCTGAAAA
GNCTTNGCNGGGCNATAGCCGGNCTATTTAAACTACGGTTCNNGGTCTCATTTGNACTGAAGCCGCNNNAAGGAGGGAGT
CGCNCGNTTGGTCGTATTTCAGAGGNNNGCTTNNCTCCACNGGCNAAACAANNATCANGNGCAAAACCGGCNGCTACTAA
TTTTTANCNNTNNNCAAANTGNGGTAAANNTNCCTCGNATNNCGAGGACNNCNNCNNTCCACATAGAGGNCACCAGGNAA
TGCCAGAGAACCGANGCTTCTNNGAAGNNGNGGNACNNANAANAAGGCTGANNCAGNANTNTNANAGNNAGACTANGTTG
GGGAGNNGANTNCNTAAAAGGAAGANCNGAANANCCCCAAGNANGTGANNNAAATNNANNAAAANNCNANCACCNNNNAG
GNNAANCGGNNGGAGGCCGACCNACNNNAAAAANNNNNNNNNNNGNNNNGNGCAGACGNNCGGGNGNGAAGGGGNGAAGA
AANNGNAANNAGANGAGGAGACGAAANAAANGGCAANNAAGGAGANAANGAGG


* A lot of Ns!

* Similar story with Ank1_56_R_blast.txt and Ank1_56_F_blast.txt

### 15/07/21

* No evidence of a deletion in vcf file. Check mapping of reads

In [2]:
import os
os.chdir("/cluster/db/mecoulter/RNAseq2/bam_all2")

samtools view -h -b HEB_124_52.bam chr3H_part1:36889000-36890300 > HEB_124_52_Ank1.bam
samtools index HEB_124_52_Ank1.bam

* Result: Deletion in intron so not picked up by RNAseq. However, reads mapping, look good, evidence of expression in 52

Now check Barke bam file... is there evidence for the pan genome annotation of the gene?

samtools view -h -b Barke.bam chr3H_part1:36880000-36890000 > Barke_NLR.bam; samtools index Barke_NLR.bam

* Tablet shows no evidence of the BaRT annotation of the NLR (but gaps between read pairs are small). Likely pan-genome annotation is correct

### Generate synthetic sequence with deletion

In [4]:
os.chdir("/cluster/db/mecoulter/Ross/")

* Parse the NLR fasta file

In [ ]:
sequence = ""
for line in open("_chr3H_36880423_36891000_sequence.fasta"):
    if line.startswith(">"):
        continue
    line = line.strip()
    #print(str(len(line)))
    sequence += line



* Delete the relevant bit of sequence

In [9]:
deletion_start = 36889835 - 36880423 + 1
deletion_end = deletion_start + 18

deletion_sequence = sequence[deletion_start:deletion_end]
print(deletion_sequence)

GTTATAGCTCCGCTATAG


In [11]:
deleted_sequence = ""

for i in range(0, len(sequence)):
    if i >= deletion_start and i < deletion_end:
        print(sequence[i])
        continue
    deleted_sequence += sequence[i]
    
print(f"Length of sequence: {len(sequence)}, length of deleted sequence {len(deleted_sequence)}")   

G
T
T
A
T
A
G
C
T
C
C
G
C
T
A
T
A
G
Length of sequence: 10578, length of deleted sequence 10560


* Use sequence to design primers (NCBI primer design)

In [13]:
import textwrap

with open("_chr3H_36880423_36891000_sequence_with52_deletion.fasta","w") as out:
    out.write(">52_NLR_deletion\n")
    out.write(textwrap.fill(deleted_sequence, width=60))
    


* Test primers

In [15]:
print(f"Forward Primer must be designed between {deletion_start - 22} and {deletion_start + 22}")

Forward Primer must be designed between 9391 and 9435


In [16]:
9391 + 22

9413

In [17]:
!cat predict_products.py

import sys
import subprocess


input_fasta = sys.argv[1]
genome = sys.argv[2]
outputfile = sys.argv[3]

#Parameters
maxproductsize = 4000
max_mismatches = 3
min_percentid = 90

#header_outfmt10 = ["name", "chromosome", "percent id", "match_length", "mismatches", "gap_openings", "Query start","Query end", "subject_start", "subject_end", "evalue","bitscore"]
class Blast10:#Different versions of blast produce different fields with outfmt10. So check
	def __init__(self,line):
		fields = line.rstrip().split(",")
		self.name = fields[0]
		self.chromosome = fields[1]
		self.percent_id = float(fields[2])
		self.match_length = int(fields[3])#Excluding gaps
		self.mismatches = int(fields[4])
		self.gap_openings = int(fields[5])
		self.query_start = int(fields[6])
		self.query_end = int(fields[7])
		self.subject_start = int(fields[8])
		self.subject_end = int(fields[9])
		self.evalue = float(fields[10])
		self.bitscore = float(fields[11])
		if self.subject_start < self.subject_end:
			self.matcht

In [18]:
!python predict_products.py 3Hlocus_deletion_primer.fasta /cluster/db/mecoulter/genomes/genomes/180903_Barke_Unfiltered_chloro_clean_pseudomolecules_v1.fasta deletion_Barke_blast_products.tab

Working on genome 180903_Barke_Unfiltered_chloro_clean_pseudomolecules_v1
Script finished!


In [21]:
import pandas as pd
table = pd.read_csv("deletion_Barke_blast_products.tab", delimiter="\t", index_col=0)
table

,chromosome,Product size,Start,End,F percent id,F match length,F query length,R percent id,R match length,R query length
Primer name,,,,,,,,,,
1,chr3H,226,36889817,36890043,100.0,19,19,100.0,22,22
7,chr3H,550,36889818,36890368,100.0,18,18,100.0,22,22


In [25]:
import subprocess

genomes = ["/cluster/db/mecoulter/genomes/genomes/Morex_pseudomolecules_v2.fasta", "/cluster/db/mecoulter/genomes/genomes/Golden_Promise_pseudomolecule_v1.fasta", "/cluster/db/mecoulter/genomes/genomes/B1K-04-12_pseudomolecules_v1.fasta" ]

for genome in genomes:
    genome_name = genome.split("/")[-1].split(".")[0]
    subprocess.run(f"python predict_products.py 3Hlocus_deletion_primer.fasta {genome} deletion_{genome_name}_blast_products.tab", shell=True)
    print(f"Finished writing deletion_{genome_name}_blast_products.tab")

Finished writing deletion_Morex_pseudomolecules_v2_blast_products.tab
Finished writing deletion_Golden_Promise_pseudomolecule_v1_blast_products.tab
Finished writing deletion_B1K-04-12_pseudomolecules_v1_blast_products.tab


In [26]:
table = pd.read_csv("deletion_Morex_pseudomolecules_v2_blast_products.tab", delimiter="\t", index_col=0)
table

,chromosome,Product size,Start,End,F percent id,F match length,F query length,R percent id,R match length,R query length
Primer name,,,,,,,,,,


In [27]:
table = pd.read_csv("deletion_Golden_Promise_pseudomolecule_v1_blast_products.tab", delimiter="\t", index_col=0)
table

,chromosome,Product size,Start,End,F percent id,F match length,F query length,R percent id,R match length,R query length
Primer name,,,,,,,,,,
1,chr3H,226,30348302,30348528,100.0,19,19,100.0,22,22
7,chr3H,550,30348303,30348853,100.0,18,18,100.0,22,22


In [28]:
table = pd.read_csv("deletion_B1K-04-12_pseudomolecules_v1_blast_products.tab", delimiter="\t", index_col=0)
table

,chromosome,Product size,Start,End,F percent id,F match length,F query length,R percent id,R match length,R query length
Primer name,,,,,,,,,,


Primers 1 and 7 may cause false positives in Barke (but perhaps not)